In [ ]:
from tvb.simulator.models.oscillator import Generic2dOscillator
from tvb.simulator.integrators import HeunStochastic
import matplotlib.pyplot as plt
import numpy as np
import math

In [ ]:
# Parameters
a_sim = 2.0
b_sim = -10.0
c_sim = 0.0
d_sim = 0.02
I_sim = 0.0
nsig = 0.003
dt = 0.1

In [ ]:
# TVB model and integrator setup
oscillator_model = Generic2dOscillator(
    a=np.asarray([a_sim]),
    b=np.asarray([b_sim]),
    c=np.asarray([c_sim]),
    d=np.asarray([d_sim]),
    I=np.asarray([I_sim]),
)
oscillator_model.configure()

integrator = HeunStochastic(dt=dt)
integrator.noise.nsig = np.array([nsig])
integrator.noise.configure()
integrator.noise.configure_white(dt=integrator.dt)
integrator.set_random_state(random_state=None)
integrator.configure()
integrator.configure_boundaries(oscillator_model)

In [ ]:
simulation_length = 300
stimulus = 0.0
local_coupling = 0.0

current_state_v = np.random.uniform(low=-2.0, high=4.0, size=[1, 1, 1])
current_state_w = np.random.uniform(low=-6.0, high=6.0, size=[1, 1, 1])
current_state = np.vstack([current_state_v, current_state_w])
state = current_state

current_step = 0
number_of_nodes = 1
start_step = current_step + 1
node_coupling = np.zeros([2, 1, 1])
n_steps = int(math.ceil(simulation_length / integrator.dt))

X = [current_state.copy()]
for step in range(start_step, start_step + n_steps):
    state = integrator.integrate(state, oscillator_model, node_coupling, local_coupling, stimulus)
    X.append(state.copy())

X = np.asarray(X)

In [ ]:
f1 = plt.figure(figsize=(14,8))
plt.plot(X[:, 0, 0, 0], label="V")
plt.plot(X[:, 1, 0, 0], label="W")
plt.xlabel("time (ms)")
plt.ylabel("states")
plt.legend()
plt.show()

In [ ]:
X.shape

In [ ]:
np.save(f"limit-cycle_simulation", X)